In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')
import panel as pn
from tqdm import tqdm
import h5py as h5
from pathlib import Path
import tables
import sys
from importlib import reload

import psana as ps
sys.path.append('/sdf/group/lcls/ds/tools/smalldata_tools/latest')
from smalldata_tools.DetObject import DetObject, DetObjectFunc
from smalldata_tools.ana_funcs.azimuthalBinning import azimuthalBinning

from smalldata_tools.SmallDataUtils import setParameter, getUserData, getUserEnvData

from smalldata_tools.utilities_plotting import hv_image_ctl
from smalldata_tools.utilities_plotting import hv_image
from smalldata_tools.utilities import image_from_dxy

run = 320
exp = 'xpptut15'

dsname = 'exp={}:run={}'.format(exp,run,exp[:3],exp)
ds = ps.MPIDataSource(dsname)

#ds.detnames()

In [ ]:
def asImg(data,ix,iy):
    if data.ndim==2: return data
    elif data.ndim==3:
        return image_from_dxy(data,ix,iy)
    elif data.ndim==4: #gain switching, slow lowest gain
        return image_from_dxy(data[0],ix,iy)

# Set detObject 

This is where the detector object is defined. Many relevant parameters for the detector are available and saved in the hdf5 file.

In [ ]:
det = DetObject('cspad', ds.env(), int(run))
userDataCfg = det.params_as_dict()

## Look at little at the detector 'configuration' information 

In [ ]:
hrms = np.histogram(userDataCfg['rms'], np.arange(0.,10,0.1))
hv.Points((hrms[1][1:], hrms[0])).options(width=800)

In [ ]:
det_rms = userDataCfg['rms']
hv_image_ctl(asImg(det_rms, userDataCfg['ix'],userDataCfg['iy']))

# Set detObject and ana function

This is where the detector and the analysis function are defined. Here we specify the 'LCLS' azimuthal averaging.
Note that userDataCfg now also contains information about the function we have define to be run on the detector data!

In [ ]:
det = DetObject('cspad', ds.env(), int(run))

func_dict = {}
func_dict['eBeam'] = 9.8
func_dict['qbin'] = 0.05
func_dict['thresADU'] = 7
#func_dict['thresADUhigh'] = 17.5
func_dict['pPlane'] = 0
func_dict['center'] = [87392.2, -10126.4]
func_dict['dis_to_sam'] = 90.
#func_dict['phiBins'] = 5
    
func = azimuthalBinning(**func_dict)

# add function to detector pipeline
det.addFunc(func)

userDataCfg = det.params_as_dict()

In [ ]:
#run this on an event

max_evt = 1
ds.break_after(max_evt) # stop iteration after max_evt events (break statements do not work reliably with MPIDataSource).

userDict = {}
for nevt,evt in tqdm(enumerate(ds.events())):
    det.getData(evt)
        
    det.processFuncs()
    userDict[det._name]=getUserData(det)

    #print(userDict)
#print(userDataCfg)

## Look at the data 
det.getData will extract the data and add it as det.evt.dat object which can be passed to DetObjectFunc objects (next noteboook)

In [ ]:
print('Detector data shape:', det.evt.dat.shape)

## Plot the detector data 

In [ ]:
det_evt_thres = det.evt.dat.copy()
thresADU = 7
det_evt_thres[det.evt.dat<thresADU]=0 



hv_image_ctl(asImg(det_evt_thres,userDataCfg['ix'],userDataCfg['iy']))

## Plot the result of the DetObjectFunc (the azimuthal average in this case)

In [ ]:
hv.Points((0.5*(userDataCfg['azav__azav_qbins'][:-1]+userDataCfg['azav__azav_qbins'][1:]),\
           userDict['cspad']['azav_azav'].sum(axis=0))).options(width=800)